In [2]:
import numpy as np
import matplotlib.pyplot as plt
from joblib import Parallel, delayed,parallel_backend

In [3]:
class Testing1():
    def __init__(self,data,model,verbose=0):
        self.data = data
        self.model = model
        self.verbose = verbose
    
    def testRow(self,row_prev,row,row_pred):
        totalMatchDynamic = 0
        totalMatchStatic = 0
        totalStatic = 0
        totalDynamic = 0
        for cell0,cell1,cell2 in zip(row_prev,row,row_pred):
            totalMatchDynamic += (cell0 != cell1 and cell1 == cell2)
            totalMatchStatic += (cell0 == cell1 and cell1 == cell2)
            totalStatic += (cell0 == cell1)
            totalDynamic += (cell0 != cell1)
        totalStatic = totalStatic if totalStatic > 0 else 0
        totalDynamic = totalDynamic if totalDynamic > 0 else 0
        return [totalStatic,totalDynamic,totalMatchStatic,totalMatchDynamic]
            
    def run(self):
        res = []
        backend = "threading"
        with Parallel(n_jobs=4,backend=backend) as parallel:
            if self.verbose >= 1: print("Testing:")
            for i in range(1,len(data)):
                if self.verbose >= 1: print(f"state {i}/{len(data)}")
                prediction = next(model.run(data[i-1],1))
                rowsTotal = parallel(delayed(self.testRow)(row0,row1,row2) for row0,row1,row2 in zip(data[i-1],data[i],prediction[1]))
                summation = np.sum(rowsTotal,axis=0)
                if self.verbose >= 1: 
                    print(f"Static : {summation[2]}/{summation[0]}")
                    print(f"Dynamic : {summation[3]}/{summation[1]}")
                totalStatic = summation[2]/summation[0] if summation[0] > 0 else 0
                totalDynamic = summation[3]/summation[1] if summation[1] > 0 else 0
                res.append([totalStatic,totalDynamic])
        self.result = np.array(res)
    
    def plot(self):
        plt.plot(range(len(self.result)),self.result[:,0])
        plt.plot(range(len(self.result)),self.result[:,1])
        return plt

In [ ]:
class CAModel():
    def __init__(self,rules,neighborhood,paddingType,paddingValue):
        self.paddingType = 1
        self.paddingValue = -1
        self.rules = rules
        self.neighborhood = neighborhood
        
    def run(self,initialState,noSteps):
        size = np.shape(initialState)
        initialState = initialState.reshape(-1)
        currentStep = copy(initialState)
        for step in range(noSteps):
            nextStep = copy(currentStep)#np.zeros(shape=np.shape(currentStep))
            for cell in helpers.multiDimensionalGenerator(size):
                neighbors = helpers.getNeighbors(cell,self.neighborhood,size)
                A = helpers.getNeighborsValue(currentStep,neighbors,self.paddingType,self.paddingValue)
                for op in self.rules:
                    res = self.rules[op](A)
                    if res:
                        xi = cell
                        expr = helpers.getExpr(size)
                        tmpCell = expr(xi)
                        nextStep[tmpCell] = op
            yield step,nextStep.reshape(size)
            currentStep = nextStep